In [17]:
import networkx as nx
import pickle
import random
import os

Of 4k sentences,

176 fail complete graph test

Of that 26% such cases **have no edge data between them**. (~45 sentences).

These are representative 5 files, 
which have correct ground truth ids, yet no edge between them.

356107
<br>
304860
<br>
29147
<br>
19425
<br>
150138

In [18]:
# just change the filenames

graphml_file = '356107.graphml'
pickle_file = '356107.p'

In [20]:

BASE_DIR = os.getcwd()
PICKLE_DCS_FILE_NAME = os.path.join(BASE_DIR,'data','no_edge',pickle_file)
GRAPHML_FILE_NAME = os.path.join(BASE_DIR,'data','no_edge',graphml_file)

In [21]:
# defining class for pickle file 

class DCS:
    def __init__(self, sent_id, sentence):
        self.sent_id = sent_id
        self.sentence = sentence
        self.dcs_chunks = []
        self.lemmas = []
        self.cng = []

dcs_class_params = ['sentence id', 'sentence ', 'chunks', 'lemmas', 'morph class']        

In [22]:

# Double characters mapping to single characters
_dbl = dict({
    'ai' : 'E',
    'au' : 'O',
    'kh' : 'K',
    'gh' : 'G',
    'ch' : 'C',
    'jh' : 'J',
    'ṭh' : 'W',
    'ḍh' : 'Q',
    'th' : 'T',
    'dh' : 'D',
    'ph' : 'P',
    'bh' : 'B'})

# One to one mapping
_oth = dict({
    'ā' : 'A',
    'ī' : 'I',
    'ū' : 'U',
    'ṛ' : 'f',
    'ṝ' : 'F',
    'ḷ' : 'x',
    'ḹ' : 'X',
    'ṃ' : 'M',
    'ḥ' : 'H',
    'ṁ' : '~',
    'ṅ' : 'N',
    'ñ' : 'Y',
    'ṭ' : 'w',
    'ḍ' : 'q',
    'ṇ' : 'R',
    'ś' : 'S',
    'ṣ' : 'z'})


In [23]:
##IAST to SLP1 transliteration


def iast2slp(src):
    '''
    Converts International Alphabet for Sanskrit Transliteration (IAST) scheme to
    Sanskrit Library Phonetic Basic notation
    '''
    tgt = ''
    inc = 0
    while inc < len(src):
        now = src[inc]
        nxt = src[inc+1] if inc < len(src) - 1 else ''
        if now + nxt in _dbl:
            tgt += _dbl[now + nxt]
            inc += 1
        elif now in _oth:
            tgt += _oth[now]
        else:
            tgt += now
        inc += 1
    return tgt




In [24]:
# Read the pickle file containing ground truth data



def pickle_data_give(filename):
    '''Returns the attributes of the DCS pickle file
    
    Args:
    filename (str) = name of file
    
    Returns:
    output_load.sent_id (int): sentence id
    output_load.sentence (str): sentence 
    output_load.dcs_chunks (list): chunks
    output_load.lemmas (list): lemmas
    output_load.cng (list): morphological class
    
    Raises:
    '''
    
    output_load = pickle.load(open(filename, "rb"), encoding='utf-8')
    
    
    return output_load.sent_id, output_load.sentence, output_load.dcs_chunks, output_load.lemmas, output_load.cng
    

In [25]:
# read graph ml file

graph_file = nx.read_graphml(GRAPHML_FILE_NAME)


In [26]:
# Recursively break and simplify a nested list

def simplify_nested_list(nested_list):   
    
    # requires resetting of "all_single_elements_list" variable prior to each function call
    
    if len(nested_list) >= 1 and type(nested_list) is list:
        for idx, ele in enumerate(nested_list):
            #print(idx, ele, type(ele))
            if type(ele) is str:
                
                all_single_elements_list.append([ele])
                continue
                
            else:
                simplify_nested_list(nested_list[idx])
                
        
    else:
        
        all_single_elements_list.append(nested_list)
        return (nested_list)
        


In [27]:
# algorithm to find the cng+lemma combinations

_, dcs_sentence, _, lemmas, cng = pickle_data_give(PICKLE_DCS_FILE_NAME)
#print(lemma, '\n',cng)

#print(len(dcs_sentence.strip().rstrip().split(' ')))
#print(dcs_sentence.strip().rstrip().split(' '))
graph_ground_truth_ids = []
flattened_chunk_no = []
print('==========================================================================')


#get chunk no's flattened list
for idx, lemma in enumerate(lemmas):
    #print(len(lemma))
    if len(lemma)>1:
        #print('$')
        tmp=[]
        for x in range(len(lemma)):
            tmp.append(str(idx+1))
        flattened_chunk_no.append(tmp)
    else:
        #print('%')
        flattened_chunk_no.append(str(idx+1))
        

all_single_elements_list = []
simplify_nested_list(flattened_chunk_no)
flattened_chunk_no = all_single_elements_list 
        
        
all_single_elements_list = []
simplify_nested_list(lemmas)
lemmas = all_single_elements_list


all_single_elements_list = []
simplify_nested_list(cng)
cng = all_single_elements_list

print('\n\nlemmas:\t%s \ncngs:\t%s' %(lemmas, cng))

lemma_cng = zip(lemmas, cng)

number_of_chunks = len(dcs_sentence.strip().rstrip().split(' '))


for dcs_lemma, dcs_cng in lemma_cng:
    print('\n\n******************************')
    print('\n\nCurrent lemma: %s \tCurrent cng:%s' %(dcs_lemma, dcs_cng))
    
    same_ground_truth_ids = []
    
    for di in graph_file.nodes():
        
        graph_file.add_node(str(di), ground_truth_id=0)
        
        # this is random one time traversal search in the whole graph
        #print('Current node:%s' %(di))
        temp_dict = graph_file.nodes[di]
        
        # convert the lemma to slp1
        dcs_lemma[0] = iast2slp(dcs_lemma[0])
        
        
        
        print('checking node %s' %(di))
        print('\t\tLemma:%s\tCng:%s\n' %(temp_dict['lemma'], temp_dict['cng']))
        
        # check if in the current node, the cng matches to the pickle file cng
        if temp_dict['cng'] == int(dcs_cng[0]):
                       
            
            
            # check if in the current node, the lemma matches to the pickle file lemma
            if (temp_dict['lemma'] == dcs_lemma[0]):
                
                # only if cng and the converted lemma match, then these are executed
                
                #if tempdict chunk number matches dcs chnk number
                print('\t\t\t\t\t|$| - Here in node %s' %(di))
                #graph_file.add_node(di, ground_truth_id=1)
                same_ground_truth_ids.append(di)
        else:
            
            graph_file.add_node(str(di), ground_truth_id=0)
        
    graph_ground_truth_ids.append(same_ground_truth_ids)
                
                #print('#', di, '\n')
    

#return len(graph_ground_truth_ids)/number_of_chunks



lemmas:	[['yad'], ['vid'], ['veda'], ['na'], ['tad'], ['vid'], ['vid'], ['satya'], ['sthā'], ['yad'], ['tu'], ['tad'], ['vid'], ['vid']] 
cngs:	[['29'], ['-153'], ['79'], ['2'], ['29'], ['-153'], ['-210'], ['171'], ['-190'], ['29'], ['2'], ['29'], ['-153'], ['-210']]


******************************


Current lemma: ['yad'] 	Current cng:['29']
checking node 4
		Lemma:vid	Cng:-153

checking node 53
		Lemma:vedya	Cng:71

checking node 11
		Lemma:veda	Cng:129

checking node 27
		Lemma:vid	Cng:-151

checking node 28
		Lemma:vid	Cng:-153

checking node 68
		Lemma:yad	Cng:29

					|$| - Here in node 68
checking node 75
		Lemma:vid	Cng:-153

checking node 90
		Lemma:vedya	Cng:69

checking node 31
		Lemma:veda	Cng:49

checking node 96
		Lemma:vedya	Cng:71

checking node 93
		Lemma:vedya	Cng:31

checking node 16
		Lemma:anna	Cng:51

checking node 21
		Lemma:na	Cng:2

checking node 71
		Lemma:tad	Cng:29

checking node 92
		Lemma:vedya	Cng:71

checking node 6
		Lemma:vid	Cng:-153

checking node


checking node 90
		Lemma:vedya	Cng:69

checking node 31
		Lemma:veda	Cng:49

checking node 96
		Lemma:vedya	Cng:71

checking node 93
		Lemma:vedya	Cng:31

checking node 16
		Lemma:anna	Cng:51

checking node 21
		Lemma:na	Cng:2

checking node 71
		Lemma:tad	Cng:29

checking node 92
		Lemma:vedya	Cng:71

checking node 6
		Lemma:vid	Cng:-153

checking node 15
		Lemma:vid	Cng:-151

checking node 14
		Lemma:vid	Cng:-153

checking node 20
		Lemma:ad	Cng:30

checking node 65
		Lemma:satya	Cng:50

checking node 84
		Lemma:vedya	Cng:71

checking node 35
		Lemma:vedya	Cng:31

checking node 44
		Lemma:vid	Cng:-210

checking node 66
		Lemma:sTA	Cng:-190

					|$| - Here in node 66
checking node 55
		Lemma:vedya	Cng:69

checking node 22
		Lemma:tad	Cng:29

checking node 10
		Lemma:veda	Cng:49

checking node 36
		Lemma:vedya	Cng:69

checking node 101
		Lemma:vedya	Cng:31

checking node 67
		Lemma:sTita	Cng:29

checking node 82
		Lemma:vedya	Cng:31

checking node 81
		Lemma:vedya	Cng:71

checking no

In [28]:
# this function is used to pick one cng+lemma combination if redundant units are present.

def pick_one_cng_lemma(graph_ground_truth_ids):
    
    update_ground_truth_ids = []
    graph_ground_truth_ids = list(filter(None, graph_ground_truth_ids))
    for ids_list in graph_ground_truth_ids:
        if len(ids_list) > 1:
            chose_cng_lemma = random.choice(ids_list)
            graph_file.add_node(chose_cng_lemma, ground_truth_id=1)
        else:
            chose_cng_lemma = ids_list[0]            
            graph_file.add_node(chose_cng_lemma, ground_truth_id=1)
            
        update_ground_truth_ids.append(chose_cng_lemma)
    
    return update_ground_truth_ids
    
graph_ground_truth_ids = pick_one_cng_lemma(graph_ground_truth_ids)
#print(graph_ground_truth_ids)

In [29]:
# check if the set of ground truth id's form a complete graph or not


def check_complete_graph(graph_file, graph_ground_truth_ids):
    graph_status = False
     
    #print(graph_file.get_edge_data(graph_ground_truth_ids[0], graph_ground_truth_ids[1])['key'])
    for idx in range(1, len(graph_ground_truth_ids)):
               
        try :
            if graph_file.get_edge_data(graph_ground_truth_ids[idx-1], graph_ground_truth_ids[idx])['key'] == 2:
                print('conflicting edge!!  \t\t\t', graph_file.has_edge(graph_ground_truth_ids[idx-1], graph_ground_truth_ids[idx]))
                graph_status = False
                break
            if graph_file.get_edge_data(graph_ground_truth_ids[idx-1], graph_ground_truth_ids[idx])['key'] == 1:
                print('not a conflicting edge \t\t\t', graph_file.has_edge(graph_ground_truth_ids[idx-1], graph_ground_truth_ids[idx]))
                print(graph_file.nodes[graph_ground_truth_ids[idx-1]]['word'], graph_file.nodes[graph_ground_truth_ids[idx]]['word'])
                print(graph_ground_truth_ids[idx-1], graph_ground_truth_ids[idx])
                print('\n')
                graph_status = True
        except KeyError and TypeError as key_type_issue:
            print(key_type_issue)
        
    return graph_status

#### The ground truth ids are given below and we check if they have a non conflicting edge or not between them.

In [30]:
#graph ground truth ids

graph_ground_truth_ids


['1',
 '77',
 '12',
 '21',
 '22',
 '6',
 '52',
 '60',
 '66',
 '1',
 '69',
 '23',
 '28',
 '87']

The below line is used to check if the nodes have edge between them or not.

In [31]:

#graph_file.has_edge('83', '86')
# uncomment the lines

The below line is used to check if they confict or not

if 2 is returned then it's a conflicting edge

if 1 is returned then it's a not a conflicting edge

In [15]:
#graph_file.get_edge_data('1', '12')

In [32]:
# snippet to get the ground truth sentence from the graphML file

print('\n\nGraph ground truth nodes:')
#graph_ground_truth_ids = sorted(graph_ground_truth_ids, key = lambda x : graph_file.nodes[x]['chunk_no'])
print(graph_ground_truth_ids)


for ids in graph_ground_truth_ids:
    ids = str(ids)
    print(graph_file.nodes[str(ids)]['word'], end=' ')

  
print('\n\n')
print('DCS sentence below:\n')

for idx, _ in enumerate(pickle_data_give(PICKLE_DCS_FILE_NAME)):
    print('%s: \t%s' %(dcs_class_params[idx], _))
    
print('\n\n\n========================')
graph_complete_or_not = check_complete_graph(graph_file, graph_ground_truth_ids)
print('\n\n\nGraph status: ',graph_complete_or_not)



Graph ground truth nodes:
['1', '77', '12', '21', '22', '6', '52', '60', '66', '1', '69', '23', '28', '87']
yaH veda vedAn na sa veda vedyam satye sTitaH yaH tu sa veda vedyam 


DCS sentence below:

sentence id: 	356107
sentence : 	yo veda vedAnna sa veda vedyaM satye sTito yastu sa veda vedyam   
chunks: 	['yad', 'vid', 'veda', 'na', 'tad', 'vid', 'vid', 'satya', 'sthā', 'yad', 'tu', 'tad', 'vid', 'vid']
lemmas: 	[['yad'], ['vid'], ['veda', 'na'], ['tad'], ['vid'], ['vid'], ['satya'], ['sthā'], ['yad', 'tu'], ['tad'], ['vid'], ['vid']]
morph class: 	[['29'], ['-153'], ['79', '2'], ['29'], ['-153'], ['-210'], ['171'], ['-190'], ['29', '2'], ['29'], ['-153'], ['-210']]



not a conflicting edge 			 True
yaH veda
1 77


not a conflicting edge 			 True
veda vedAn
77 12


not a conflicting edge 			 True
vedAn na
12 21


not a conflicting edge 			 True
na sa
21 22


not a conflicting edge 			 True
sa veda
22 6


'NoneType' object is not subscriptable
'NoneType' object is not subscriptabl

As you can see in the above output in certain edge connections, **it show's
'NoneType' object is present.**

**_There is no edge weight between such nodes, to know whether they are conflicting or not._**

### Possible solutions from here

Same as the ones mentioned in the conflict_edge jupyter notebook